In [1]:
import json
from pprint import pprint


In [115]:

import pandas as pd
from datetime import datetime
from enum import Enum
import sys

In [116]:
a = open('louis.json')


In [117]:
#'KR is for Korea
#4 for derivatives
#'Then type 1 for future ,2 for call, 3 for put
#'0 no clue
#'Then 1 for maxi or 5 for mini
#'Then 2 letters expiry code, 1 one is year, second month
#'Then 3 digits for the strike
#'Then 1 for isin checksum


In [118]:
data = json.load(open('louis.json'))

In [119]:
#aPD = pd.DataFrame.from_records(data)
#books = aPD.book.drop_duplicates().str.split('01')
#books = aPD.book.drop_duplicates() #check 1 if maxi 5 if mini
#aTest = books.to_frame

In [127]:
class ContractType(Enum):
    FUTURE = 1
    OPTION = 2
    
#class Strike(Enum):
#    for i in range(50)+100:
#         str(i+2.5*i) = i-99

In [142]:
def getExpiryDate(anExpiryCode):
    #hardcoded version for now. 
    myString_year = ''
    myString_month = ''
    
    if anExpiryCode[0] == 'L':
        myString_year = "2016"
    elif anExpiryCode[0] == 'M':
        myString_year = "2017"
    elif anExpiryCode[0] == 'N':
        myString_year = "2018"
    else : 
        sys.stderr.write("fatal error\n expiry year code unknown \n")
    
    if anExpiryCode[0] == 'L':
        myString_year = "2016"
    
    if anExpiryCode[0] == 'M':
        myString_month = "2017"
    elif anExpiryCode[0] == 'N':
        myString_month = "2018"
    else : 
        sys.stderr.write("fatal error\n expiry month code unknown \n")    
    -12-20 16:00:00
    return datetime.strptime(string_date, "%Y-%m-%d %H:%M:%S")


In [143]:
from dateutil import tz
def utcToRightTMZ(aUTCTMZ):
    
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('Asia/Hong_Kong')
    
    aUTCTMZ = aUTCTMZ.replace(tzinfo=from_zone)
    localTimeStamp = aUTCTMZ.astimezone(to_zone)
    
    return localTimeStamp

In [144]:
def getBookContract(aStringNumber):
    aContract = ''
    if aStringNumber == "000":
        aContract = ContractType.FUTURE #replace with enums
    else:
        aContract = ContractType.OPTION #replace with enums
    
    return aContract

In [147]:
counter = 0

for tick in data:
    
    myTick = dict()
    
    counter = counter + 1
    myBook = tick["book"]
    if myBook[5] == '1':   #let's forget about mini futures
        print(myBook)
        timeStamp = utcToRightTMZ(datetime.utcfromtimestamp(tick["created"]/1000000))
        #print(timeStamp)
        myTick["created"] = timeStamp
        myTick["received"] = utcToRightTMZ(datetime.utcfromtimestamp(tick["received"]/1000000))
        
        myTempStrike = data[1]['book'][8:11]
        myTick["contract"] = getBookContract(myBook[8:11])
        if(myTick["contract"]  == ContractType.OPTION):
            myTick["Strike"] = myBook[8:11]
        print(myBook[6:8])
        myTick["expiry"] = getExpiryDate(myBook[6:8])

        myTick["spread"] = tick['ask_price_0']-tick['bid_price_0']
        myTick["midPoint"] = (tick['ask_price_0']+tick['bid_price_0'])/2
        if counter > 50 :
            break

KR4201MC3404
MC
KR4101N60009
N6
KR4301MC3320
MC
KR4301MC3221
MC
KR4301MC3304
MC
KR4301MC3122
MC
KR4301MC3270
MC
KR4101MC0005
MC
KR4201MC3677
MC
KR4101MC0005
MC
KR4101MC0005
MC
KR4301MC3254
MC
KR4201MC3370
MC
KR4201MC3420
MC
KR4301MC3304
MC
KR4301MC3270
MC
KR4301MC3353
MC
KR4201N13279
N1


fatal error
 expiry code unknown 


ValueError: time data '' does not match format '%Y-%m-%d %H:%M:%S'